In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import argparse
import sys; sys.argv=['']; del sys
import time


seed=200
np.random.random(seed)
torch.manual_seed(seed)

In [2]:
class HH_Dataset(torch.utils.data.Dataset):
    
    def __init__(self,root_dir,X,Y,train=True):

        self.data=(X.values.astype(np.float32),Y.values.astype(int))
        self.train=train
        self.root_dir=root_dir
        self.train=train
        
    def __len__(self):
        return len(self.data[1])
    
    def __getitem__(self,idx):
        sample=(self.data[0][idx,...],self.data[1][idx])
        return sample

In [3]:
def load_data(dataset_size=None):
    data_file='datacsv3.csv'
    root_dir=''
    
    features=['x-position','x-momentum','y-position','y-momentum','Energy','Periodic']
    df=pd.read_csv(root_dir+data_file,header=None,nrows=dataset_size,engine='python')
    df.columns=features
    df.sample(frac=1)
    
    X=df[['x-position','x-momentum','y-position','y-momentum','Energy']]
    Y=df['Periodic']
    
    #Y=df['Energy']
    #X=df[['x-position','x-momentum','y-position','y-momentum']]
    
    dataset_size=X.shape[0]
            
    train_size=int(0.8*dataset_size)
    train_X=X[:train_size]
    train_Y=Y[:train_size]
    test_X=X[train_size:]
    test_Y=Y[train_size:]
    
    batch_size=int(0.01*dataset_size)
    
    test_loader=torch.utils.data.DataLoader(HH_Dataset(root_dir,test_X,test_Y,train=False),batch_size=1000,shuffle=True)
    train_loader=torch.utils.data.DataLoader(HH_Dataset(root_dir,train_X,train_Y),batch_size=batch_size,shuffle=True)
    
    return train_loader, test_loader

In [10]:
class model(nn.Module):
    def __init__(self,neurons):
        super(model,self).__init__()
        
        self.mid_neurons=neurons
        
        self.fc1=nn.Linear(5,self.mid_neurons)
        #self.fc1=nn.Linear(4,self.mid_neurons)
        self.fc2=nn.Linear(self.mid_neurons,200)
        self.fc3=nn.Linear(200,2)
        
        #self.fc3=nn.Linear(200,100)
        #self.fc4=nn.Linear(100,1)
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.dropout(x,training=self.training)
        x=F.relu(self.fc2(x))
        x=F.dropout(x,training=self.training)
        x=self.fc3(x)
        x=F.log_softmax(x,dim=1)
        
        #x=F.relu(x)
        #x=self.fc4(x)
        
        return x

In [5]:
def evaluate_model(args, train_loader, test_loader):
    DNN=model(args.neurons)
    criterion=F.nll_loss
    optimizer=args.optimizer(DNN.parameters(),lr=args.lr,weight_decay=args.l2)

    
    def train(epoch):
        DNN.train()
        for batch_idx, (data,label) in enumerate(train_loader):
            optimizer.zero_grad()
            output=DNN(data)
            
            label=label.type(torch.LongTensor)
            loss=criterion(output,label)
            
            loss.backward()
            optimizer.step()
            
            if batch_idx%10==0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item() ))
                
        return loss.item()
    
    def test():
        DNN.eval()
        test_loss=0
        correct=0
        
        for data,label in test_loader:
            output=DNN(data)
            label=label.type(torch.LongTensor)
            test_loss+=criterion(output,label).item()
            pred=output.data.max(1,keepdim=True)[1]
            correct+=pred.eq(label.data.view_as(pred)).cpu().sum().item()
        
        test_loss /= len(test_loader.dataset)
        
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))
        

        return test_loss, correct / len(test_loader.dataset)
        
    
    train_loss=np.zeros((10,))
    test_loss=np.zeros_like(train_loss)
    test_accuracy=np.zeros_like(test_loss)
    
    epochs=range(1,11)
    for epoch in epochs:
        train_loss[epoch-1]=train(epoch)
        test_loss[epoch-1],test_accuracy[epoch-1]=test()
            
    return test_loss[-1],test_accuracy[-1]

In [6]:
def grid_search(args):
    middle_neurons=[50,100,200,400]
    optimizers=[optim.SGD,optim.Adam,optim.RMSprop,optim.Adagrad]
    optimizer_name=['SGD','ADAM','RMSPROP','ADAGRAD']
    lr=[10**(-3),10**(-4),10**(-2),10**(-2)]
    l2_regularizer=np.logspace(-4,0,5)
    
    test_loss=np.zeros((len(middle_neurons),len(optimizers),l2_regularizer.shape[0]))
    test_accuracy=np.zeros_like(test_loss)
    
    train_loader,test_loader=load_data(1200000)
    
    for i,optimizer in enumerate(optimizers):
        args.optimizer=optimizer
        for j,neurons in enumerate(middle_neurons):
            args.neurons=neurons
            for k,l2 in enumerate(l2_regularizer):
                args.l2=l2
                print("\n training DNN with %d neurons and "%(neurons)+optimizer_name[i]+" with l2 regularizer = %0.6f"%(l2))
                test_loss[i,j,k],test_accuracy[i,j,k]=evaluate_model(args,train_loader,test_loader)
                
        plot_data(l2_regularizer,middle_neurons,test_accuracy[i,:,:],optimizer_name[i])

In [7]:
def plot_data(x,y,data,name):
        # plot results
    fontsize=16


    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(data, interpolation='nearest', vmin=0, vmax=1)
    
    cbar=fig.colorbar(cax)
    cbar.ax.set_ylabel('accuracy (%)',rotation=90,fontsize=fontsize)
    cbar.set_ticks([0,.2,.4,0.6,0.8,1.0])
    cbar.set_ticklabels(['0%','20%','40%','60%','80%','100%'])

    # put text on matrix elements
    for i, x_val in enumerate(np.arange(len(x))):
        for j, y_val in enumerate(np.arange(len(y))):
            c = "${0:.1f}\\%$".format( 100*data[j,i])  
            ax.text(x_val, y_val, c, va='center', ha='center')

    # convert axis vaues to to string labels
    x=[str(i) for i in x]
    y=[str(i) for i in y]


    ax.set_xticklabels(['']+x)
    ax.set_yticklabels(['']+y)

    ax.set_xlabel('$\\mathrm{l2\\ regularizer}$',fontsize=fontsize)
    ax.set_ylabel('$\\mathrm{number\\ of\\ neurons\\ middle\\ layer}$',fontsize=fontsize)
    ax.set_title(name)

    plt.tight_layout()
    fig.savefig('HH_dataset_'+name+'.png')

    plt.show()

In [8]:
parser=argparse.ArgumentParser(description='HenonHeiles DNN')
parser.add_argument('--neurons',type=int,default=200,metavar='MN')
parser.add_argument('--optimizer',type=object,default=optim.SGD,metavar='OPT')
parser.add_argument('--l2',type=float,default=10**(-3),metavar='L2')
parser.add_argument('--lr',type=float,default=10**(-3),metavar='LR')
args=parser.parse_args()

In [ ]:
start_t=time.time()
grid_search(args)
end_t=time.time()

print(end_t-start_t)


 training DNN with 50 neurons and SGD with l2 regularizer = 0.000100
Train Epoch: 1 [0/960000 (0%)]	Loss: 0.723865
Train Epoch: 1 [120000/960000 (12%)]	Loss: 0.706938
Train Epoch: 1 [240000/960000 (25%)]	Loss: 0.689732
Train Epoch: 1 [360000/960000 (38%)]	Loss: 0.673882
Train Epoch: 1 [480000/960000 (50%)]	Loss: 0.658310
Train Epoch: 1 [600000/960000 (62%)]	Loss: 0.642472
Train Epoch: 1 [720000/960000 (75%)]	Loss: 0.629162
Train Epoch: 1 [840000/960000 (88%)]	Loss: 0.614160

Test set: Average loss: 0.0006, Accuracy: 240000/240000 (100.000%)

Train Epoch: 2 [0/960000 (0%)]	Loss: 0.600413
Train Epoch: 2 [120000/960000 (12%)]	Loss: 0.586570
Train Epoch: 2 [240000/960000 (25%)]	Loss: 0.573299
Train Epoch: 2 [360000/960000 (38%)]	Loss: 0.560743
Train Epoch: 2 [480000/960000 (50%)]	Loss: 0.549457
Train Epoch: 2 [600000/960000 (62%)]	Loss: 0.537311
Train Epoch: 2 [720000/960000 (75%)]	Loss: 0.525444
Train Epoch: 2 [840000/960000 (88%)]	Loss: 0.514446

Test set: Average loss: 0.0005, Accuracy